In [20]:
#Importing libraries

import pandas as pd
import glob

#retrieving all data files from folder
files = glob.glob(r"C:\Users\allan\OneDrive\Desktop\Documentos\Mis Análsis\Mercamadrid\data\volpre*.csv")


dfs = [pd.read_csv(f, encoding="ISO-8859-1", sep=';', decimal=',') for f in files]
df=pd.concat(dfs, ignore_index=True)

#Checking initial structure
print (df.head())
print (df.info())

  Fecha Desde  Fecha Hasta  Código Variedad             Descripción Variedad  \
0    20210101     20210131          CA 0101   VACUNO CANAL                     
1    20210101     20210131          CA 0101   VACUNO CANAL                     
2    20210101     20210131          CA 0101   VACUNO CANAL                     
3    20210101     20210131          CA 0101   VACUNO CANAL                     
4    20210101     20210131          CA 0101   VACUNO CANAL                     

   Origen     Descripción Origen       Kilos  Precio Más Frecuente  \
0     005   AVILA                  000002220                  4,27   
1     008   BARCELONA              000066910                  4,29   
2     010   CACERES                001327305                  4,29   
3     015   LA CORUÑA              000052640                   4,3   
4     017   GERONA                 000010440                  4,28   

   Precio Máximo  Precio Mínimo  
0           4,27           4,27  
1           4,31           4,2